# Installing h3 Lib

In [ ]:
!pip install ../input/h3lib/h3-3.7.4-cp37-cp37m-manylinux_2_17_x86_64.manylinux2014_x86_64.whl

# Limporting lib

In [ ]:
import difflib
from h3 import h3
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from unidecode import unidecode
from tqdm.notebook import tqdm

tqdm.pandas()
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))



# Data importing (train and test sets)

In [ ]:
df_train = pd.read_csv('../input/training-set-foursquare/training_set.csv')
training_set_ = df_train.copy()

df_test = pd.read_csv('/kaggle/input/foursquare-location-matching/test.csv')
test_set_ = df_test.copy()

# Data first check

In [ ]:
training_set_.info()

In [ ]:
missing = training_set_.isna().sum()
missing

In [ ]:
# normalizing name to avoid mismatch due to accents or punctuations
test_set_['name'] = test_set_['name'].astype(str)
test_set_['name'] = test_set_['name'].apply(unidecode)
test_set_['name'] = test_set_['name'].str.replace('[^\w\s]','')
test_set_['name'] = test_set_['name'].str.lower()

# normalizing categories to avoid mismatch due to accents or punctuations
test_set_['categories'] = test_set_['categories'].astype(str)
test_set_['categories'] = test_set_['categories'].apply(unidecode)
test_set_['categories'] = test_set_['categories'].str.replace('[^\w\s]','')
test_set_['categories'] = test_set_['categories'].str.lower()

# Feature Engineer

## So, considering the number of categories available, it was unviable to classify all of them by myself. After some research, I found the LDA method that allows to cluster of words into categories. (check: https://towardsdatascience.com/topic-modelling-in-python-with-nltk-and-gensim-4ef03213cd21).



In [ ]:
import re
import unidecode

def tokenization(text):
    tokens = re.split(' ',text)
    return tokens

test_set_['category_tokenied']= test_set_['categories'].apply(lambda x: tokenization(x))
training_set_['category_tokenied']= training_set_['categories'].astype(str).apply(lambda x: tokenization(x))

data = training_set_.category_tokenied.values.tolist()

In [ ]:
import gensim.corpora as corpora # Create Dictionary

id2word = corpora.Dictionary(data) # Create Corpus

texts = data # Term Document Frequency

corpus = [id2word.doc2bow(text) for text in texts]# View


In [ ]:
import gensim
from gensim.utils import simple_preprocess
from pprint import pprint# number of topics

num_topics = 4 # from 3, 4, 5, 10 ,15 - 4 clusters was the one that was the best coherence

lda_model = gensim.models.LdaMulticore(corpus=corpus,
                                       id2word=id2word,
                                       num_topics=num_topics,
                                       random_state=10)# Print the Keyword in the 10 topics
pprint(lda_model.print_topics())
doc_lda = lda_model[corpus]

## Let's check how to get the cluster from this model

In [ ]:
def prepare_text_for_lda(text):
    tokens = tokenization(text)
    tokens = [token for token in tokens if len(token) > 4]
    return tokens


## So get_document_topics returns a list of tuples with the percent of similarity with each cluster. Let's use it to apply for all the row by definig a function and them apply it. 

In [ ]:
def get_lda_cluster(category):
    new_doc = prepare_text_for_lda(category)
    new_doc_bow = id2word.doc2bow(new_doc)
    clusters = lda_model.get_document_topics(new_doc_bow)
    
    scores = []
    for i in clusters:
        scores.append(i[1])

    max_index = scores.index(max(scores))
    return clusters[max_index][0]
    

In [ ]:
test_set_['category_cluster'] = test_set_.progress_apply(lambda x: get_lda_cluster(x['categories']),axis=1)


### My approach was using the h3 lib created by Uber (check documentation here: https://github.com/uber/h3) to cluster POIs into hex using the method h3.geo_to_h3 first. It will help us to check in a certain hex all the places.  
### It allows to search new POIs according the hex_cell id which combined to others features to solve eventually conflits, can solve this bussiness problem. 

In [ ]:
test_set_.rename(columns = {'longitude': 'lng', 'latitude': 'lat'}, inplace = True)

## Creating the Hex_cell ids

In [ ]:
H3_res = 9
def geo_to_h3(row):
  return h3.geo_to_h3(lat=row.lat,lng=row.lng,resolution = H3_res)

In [ ]:
test_set_['h3_cell'] = test_set_.progress_apply(geo_to_h3,axis=1)

# Defining the search engine 

### Here I'm going to use all the features that I created to build a engine search. First i'm going to use the hex id to filter all the places of a certain area, than I will filter all of them by the category cluster. Than, to solve eventually conflits, I'm using the difflib.get_close_matches to get the most similar name str. 

In [ ]:
def search_engine(id, name, h3_cell, cluster):
    first_step = training_set_[training_set_['h3_cell'] == h3_cell]
    second_step = first_step[training_set_['category_cluster'] == cluster]
    
    kw = difflib.get_close_matches(name, second_step['name'], n=3)
    
    result = second_step[second_step['name'].isin(kw)]
    if result['id'].empty:
        return id
    else:
        sep = ' '
        match_formated = id
        for match in result['id']:
            match_formated += sep 
            match_formated += match
        return match_formated

In [ ]:
test_set_['matches'] = test_set_.apply(lambda x: search_engine(x['id'], x['name'], x['h3_cell'], x['category_cluster']),axis=1)

In [ ]:
test_set_

In [ ]:
training_set_[training_set_['id'] == 'E_161706a74b5308']

In [ ]:
training_set_[training_set_['id'] == 'E_6e477dd29c2dc2']

## As we can see, it was enough to find a pretty good match for two of 5 rows of the sample

In [ ]:
submission_sample = test_set_[['id','matches']]


In [ ]:
submission_sample

# Sample Submittion

In [ ]:
submission_sample.to_csv('/kaggle/working/submission.csv')